In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

from keras.optimizers import Nadam
from keras import regularizers

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, TimeDistributed
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from keras import optimizers

import joblib

Upload data (someway, depending on your data).
I'll assume they're encoded in a pandas DataFrame.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
fd = pd.read_pickle("/content/drive/My Drive/total.pickle")

In [4]:
print('samples x dimensions x nchannels x npoints ', fd.shape)
print('dimensions x nchannels x npoints ',fd[0].shape)
print('nchannels x npoints ', fd[0][0].shape)
print('npoints', fd[0][0][0].shape)

samples x dimensions x nchannels x npoints  (1000, 2, 4, 79)
dimensions x nchannels x npoints  (2, 4, 79)
nchannels x npoints  (4, 79)
npoints (79,)


Divide the data in training (validation) and test.
In my dataset the data had already been shuffled.

In [5]:
fd_train = np.stack((np.array([sample for sample in fd[:,0,0,:]])[:491], (fd[:491,1,0,:])), axis=-1)
fd_test = np.stack((np.array([sample for sample in fd[:,0,0,:]])[491:], (fd[491:,1,0,:])), axis=-1)

In [6]:
x_amplitude = fd_train[:,:,0]
x_phase=fd_train[:,:,1]

Define the model

In [19]:
n_features=fd_train.shape[1]
time_window=79

In [20]:
def baseline_model(dropout=0.4, bias1=1e-9, bias2=1e-9, ker1=1e-9, ker2=1e-9):
  
  model = Sequential()  
  model.add(Dense(int(n_features/2), activation='tanh', input_shape=(time_window, 2), bias_regularizer=regularizers.l1_l2(l1=bias1, l2=bias2), kernel_regularizer=regularizers.l1_l2(l1=ker1, l2=ker2)))
  model.add(Dropout(dropout))

  model.add(Dense(int(n_features/4), activation='tanh', input_shape=(time_window, 2), bias_regularizer=regularizers.l1_l2(l1=bias1, l2=bias2), kernel_regularizer=regularizers.l1_l2(l1=ker1, l2=ker2)))  
  model.add(Dropout(dropout))

  model.add(Dense(int(n_features/2), activation='tanh', bias_regularizer=regularizers.l1_l2(l1=bias1, l2=bias2), kernel_regularizer=regularizers.l1_l2(l1=ker1, l2=ker2)))
  model.add(Dense(int(n_features), activation='linear', bias_regularizer=regularizers.l1_l2(l1=bias1, l2=bias2), kernel_regularizer=regularizers.l1_l2(l1=ker1, l2=ker2)))
  model.add(TimeDistributed(Dense(2)))
  
  model.compile(optimizer=Nadam(lr=5e-6), loss='mse', metrics=['mse'])

  return model

In [25]:
mlp = KerasRegressor(build_fn=baseline_model, epochs=1, batch_size=n_features, verbose=0)

In [26]:
param_distr = dict(bias1 = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1], 
                   bias2 = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
                   ker2 = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1], 
                   ker1 = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
                   dropout = [1e-1, 2e-1, 3e-1, 4e-1, 5e-1])

In [27]:
grid_search = GridSearchCV(estimator = mlp, param_grid=param_distr, cv=3)

In [ ]:
grid_search.fit(fd_train, fd_train)

In [ ]:
print("Best: %f using %s" % (grid_search.best_score_, grid_search.best_params_))
means  = grid_search.cv_results_['mean_test_score']
stds   = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']

Save the model

In [ ]:
joblib.dump(grid_search.best_params_, 'best_model.pkl')